In [8]:
import zipfile
import shutil
import os
import sys
import yaml


In [9]:

class Normalization:
    def __init__(self):

        self.normalization_type = 'default'

        self.position = {'minPos': -120, 'maxPos': 120}


        self.distance = {'minDistance': 0, 'maxDistance': 6}

        self.speed = {'minSpeed': -15, 'maxSpeed': 15}
        
        self.radius = {'minRad': 0, 'maxRad': 2}


    def to_dict(self):
        return self.__dict__


#########################

class Feature:
    def __init__(self):
        self.distGraph = 6
        self.nbHist = 4
        self.nbRolloutOut = 10
        self.output = 'speed'
        self.inShape = 8
        self.edgeShape = 5

    def to_dict(self):
        return self.__dict__


###########################

class SimulationParameters:
    def __init__(self):
        self.noisy = 0
        self.nMin = 120
        self.nMax = 300
        self.v0 = 60
        self.k = 70
        self.epsilon = 0.5
        self.tau = 3.5
        self.T = 1000
        self.dt = 0.001
        self.threshold = 6
        self.R = 1
        self.boundary = 120


    def to_dict(self):
        return self.__dict__

class Simulation:
    def __init__(self):
        self.nbSimLearning = 1000
        self.nbValidation = 20
        self.nbTest = 10
        self.initialization = 'easy'        # random, easy, circle
        self.initDistance = 7
        self.parameters = SimulationParameters()


    def to_dict(self):
        data = self.__dict__.copy()
        data['parameters'] = self.parameters.to_dict()
        return data


#############################

class DataAugment:
    def __init__(self):
        self.bool = 1
        self.prob = 0.8
        self.stdDeltaPos = 2
        self.stdSpeed = 0.003

    def to_dict(self):
        return self.__dict__

class Loss:
    def __init__(self):
        self.topk = -1
        self.lossScaling = 100
        self.l1Reg = 0.001
        self.lim = 35
        self.lambdaL2Weights = 0.00005


    def to_dict(self):
        return self.__dict__

class SchedulerExp:
    def __init__(self):
        self.gamma = 0.9

    def to_dict(self):
        return self.__dict__

class SchedulerLinear:
    def __init__(self):
        self.size = 1
        self.gamma = 0.5

    def to_dict(self):
        return self.__dict__

class Training:
    def __init__(self):
        self.nbEpoch = 3000
        self.modelName = 'simplest_dropout_no-encoder'
        self.evalModel = 'simplest_drop_no-enc_aug_best.pt'
        self.saveModel = 'simplest_drop_no-enc_aug_latest.pt'
        self.pathData = '/scratch/users/jpierre/mew_0.001_normal'
        self.pathJsonBool = True
        self.wbName = 'Simplest_normal_0.001-lr_0_005_b-32'
        self.rolloutNb = 4
        self.batch = 16
        self.batchSizeVal = 128
        self.lr = 0.005
        self.dataAugment = DataAugment()
        self.loss = Loss()
        self.scheduler = 'exp'
        self.scheduler_exp = SchedulerExp()
        self.scheduler_linear = SchedulerLinear()
        self.frequenceEval = 50
        self.frequenceSave = 5000

    def to_dict(self):
        data = self.__dict__.copy()
        data['dataAugment'] = self.dataAugment.to_dict()
        data['loss'] = self.loss.to_dict()
        data['scheduler_exp'] = self.scheduler_exp.to_dict()
        data['scheduler_linear'] = self.scheduler_linear.to_dict()
        return data

class Config:
    def __init__(self):
        self.normalization = Normalization()
        self.feature = Feature()
        self.simulation = Simulation()
        self.training = Training()

    def to_dict(self):
        return {
            'normalization': self.normalization.to_dict(),
            'feature': self.feature.to_dict(),
            'simulation': self.simulation.to_dict(),
            'training': self.training.to_dict()
        }


def createYaml(data, f):

    with open(f'{f}.yml', 'w') as file:
        yaml.dump(data, file, default_flow_style=False)

import os
def makedir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [10]:
cfg = Config()

createYaml(cfg.to_dict(), 'master/code/test_zone/test')

In [41]:
BASE_FILE = '/master/code/test_zone/exp-sum'
BASH_FOLDER = '/master/code/bash_script'
FUN_FOLDER = '/master/code/prep_zip/base'
OUT_FOLDER = '/master/code/test_zone/out'

In [26]:
print(os.listdir(FUN_FOLDER))

['training_main.py']


In [52]:
def create_bash(path_sh, path_data, Alan_nb):
    bash_script_content = f"""#!/bin/bash

    cd /home/jpierre/v2/new_runs/{Alan_nb}/{path_data}
    conda activate myenvPy
    python training.py
    """

    with open(path_sh, 'w') as file:
        file.write(bash_script_content)


    st = os.stat(path_sh)
    os.chmod(path_sh, st.st_mode | 0o755)


In [53]:
def createExpReLU():

    for dataType in ['normal']:
        for dt in [0.001]:
            for lr in [0.005, 0.0005]:
                for batch in [16]:
                    for encBool in ['no-encoder']:
                        for dropBool in ['dropout', 'no-dropout']:
                            for path_data in ['classic', 'simple']:
                                for additional in ['relu-exp']:
                                
                                
                                    wb_name = f'master-thesis_{dataType}_{path_data}_dt-{dt}_lr-{lr}_batch-{batch}_encoder-{encBool}_dropout-{dropBool}_{additional}'

                                    cfg = Config()
                                    cfg.training.batch = batch
                                    cfg.training.lr = lr
                                    cfg.training.wbName = wb_name

                                    modelName = 'simplest'
                                    modelName = modelName + f'_{dropBool}'
                                    modelName = modelName + f'_{encBool}'

                                    if additional in ['relu-exp']:
                                        modelName = modelName + '-relu'

                                    cfg.training.modelName = modelName


                                    if path_data in ['classic']:
                                        cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_normal'
                                    elif path_data in ['simple']:
                                        cfg.training.pathData = '/scratch/users/jpierre/smallest_0.001_normal'

                                    new_folder = os.path.join(BASE_FILE, wb_name)
                                    makedir(new_folder)
                                    
                                    createYaml(cfg.to_dict(), os.path.join(new_folder, 'config'))

In [61]:
def createEnvs(path_base, nb_Alan, path_bash, path_functions, path_out):
    
    nb = 0      # index of the number of files in an Alan
    i = 0       # nb of total files
    
    # get all the cfgs
    folders = os.listdir(path_base)

    # get all the functions
    function_files = os.listdir(path_functions)

    # initialize the Alan folder
    p_alan = os.path.join(path_out, 'Alan')
    if not os.path.exists(p_alan):
        os.makedirs(p_alan)

    # initialize the bash folders
    path_bash = os.path.join(path_bash, 'Alan_bash')
    if not os.path.exists(path_bash):
        os.makedirs(path_bash)

    for p_b in ['Alan1', 'Alan2', 'Alan3', 'Alan4']:

        # create the folder for the Alan i (in bash and in the run place)
        p_base = os.path.join(p_alan, p_b)
        p_base_bash = os.path.join(path_bash, p_b)

        # create Alan_{i}
        if not os.path.exists(p_base):
            os.makedirs(p_base)

        if not os.path.exists(p_base_bash):
            os.makedirs(p_base_bash)

        while(i < len(folders)):
            if nb == nb_Alan:
                nb = 0
                break

            # path of the config
            p_cfg = os.path.join(path_base, folders[i])
            p_cfg = os.path.join(p_cfg, 'config.yml')

            # path of the folder with everything (output)
            p = os.path.join(p_base, folders[i])
    
            if os.path.exists(p):
                print('ISSUE HERE')

            os.makedirs(p)
            p_cfg2 = os.path.join(p, 'cfg.yml')
            shutil.move(p_cfg, p_cfg2)
            shutil.rmtree(os.path.join(path_base, folders[i]))      # get rid of the stuff that are created before

            os.makedirs(os.path.join(p, 'model_trained'))   # create folder for the models

            # copy the functions
            for fun in function_files:
                p_fun = os.path.join(path_functions,fun)
                p_fun2 = os.path.join(p,fun)

                shutil.copy(p_fun, p_fun2)


            # create the corresponding bash files
            p_bash = os.path.join(p_base_bash, f'exp_{i}.sh')
            create_bash(p_bash, folders[i], p_b)

            i += 1
            nb += 1

In [62]:
createExpReLU()

In [63]:
createEnvs(BASE_FILE, 5, BASH_FOLDER, FUN_FOLDER, OUT_FOLDER)